# Section 1-0 - Prelude

In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from time import time

np.random.seed(1337)

df = pd.read_csv('../titanic.csv')

In [2]:
df_train = df.iloc[:712, :]

df_train = df_train.drop(['Name', 'Ticket', 'Cabin', 'Embarked'], axis=1)

age_mean = df_train['Age'].mean()
df_train['Age'] = df_train['Age'].fillna(age_mean)
df_train['Sex'] = df_train['Sex'].map({'female': 0, 'male': 1}).astype(int)

scaler = StandardScaler()

X_train = scaler.fit_transform(df_train.iloc[:, 2:].values)
y_train = df_train['Survived'].values
y_train_onehot = pd.get_dummies(df_train['Survived']).values

In [3]:
df_test = df.iloc[712:, :]

df_test = df_test.drop(['Name', 'Ticket', 'Cabin', 'Embarked'], axis=1)

df_test['Age'] = df_test['Age'].fillna(age_mean)
df_test['Sex'] = df_test['Sex'].map({'female': 0, 'male': 1}).astype(int)

X_test = scaler.transform(df_test.iloc[:, 2:].values)
y_test = df_test['Survived'].values

In [4]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(random_state=0, verbose=3)
model = model.fit(X_train, y_train)

y_prediction = model.predict(X_test)
print "\naccuracy", np.sum(y_prediction == y_test) / float(len(y_test))

building tree 1 of 10
building tree 2 of 10
building tree 3 of 10
building tree 4 of 10
building tree 5 of 10
building tree 6 of 10
building tree 7 of 10
building tree 8 of 10
building tree 9 of 10
building tree 10 of 10

accuracy 0.832402234637


[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Done  10 out of  10 | elapsed:    0.0s finished


In [5]:
def softmax(x):
    return np.exp(x) / np.exp(x).sum()

In [6]:
min_loss = 1000
best_weights = ()

start = time()

for i in xrange(1000):
    W = np.random.rand(2, 6) / 10
    b = np.random.rand(2,) / 10

    scores = []
    loss = 0
    
    for j in xrange(X_train.shape[0]):
        result = np.dot(W, X_train[j]) + b
        result = softmax(result)
        scores.append(list(result))
        
        label_index = np.argmax(y_train_onehot[j])
        loss += -np.log(result[label_index])

    loss = loss / float(X_train.shape[0])
    y_prediction = np.argmax(np.array(scores), axis=1)
    accuracy = np.sum(y_prediction == y_train) / float(len(y_train))
    
    if loss < min_loss:
        min_loss = loss
        best_weights = (W, b)
        print 'loss %s accuracy %s loop %s' % (round(loss, 3), round(accuracy, 3), i)

print '\ntime taken %s seconds' % str(time() - start)

loss 0.726 accuracy 0.292 loop 0
loss 0.685 accuracy 0.64 loop 1
loss 0.683 accuracy 0.642 loop 2
loss 0.681 accuracy 0.695 loop 6
loss 0.675 accuracy 0.712 loop 9
loss 0.673 accuracy 0.737 loop 45
loss 0.661 accuracy 0.739 loop 48
loss 0.658 accuracy 0.715 loop 298
loss 0.652 accuracy 0.77 loop 515

time taken 9.79911994934 seconds


In [7]:
W, b = best_weights
scores = []

for j in xrange(X_test.shape[0]):
    result = np.dot(W, X_test[j]) + b
    result = softmax(result)
    scores.append(list(result))

y_prediction = np.argmax(np.array(scores), axis=1)

print "accuracy", np.sum(y_prediction == y_test) / float(len(y_test))

accuracy 0.810055865922


In [8]:
min_loss = 1000
best_weights = ()

start = time()

for i in xrange(1000):
    W_1 = np.random.rand(100, 6) / 10
    b_1 = np.random.rand(100,) / 10
    W_2 = np.random.rand(2, 100) / 10
    b_2 = np.random.rand(2,) / 10
    
    scores = []
    loss = 0

    for j in xrange(X_train.shape[0]):
        result = np.dot(W_1, X_train[j]) + b_1
        result = np.dot(W_2, result) + b_2
        result = softmax(result)
        scores.append(list(result))
        
        label_index = np.argmax(y_train_onehot[j])
        loss += -np.log(result[label_index])

    loss = loss / float(X_train.shape[0])
    y_prediction = np.argmax(np.array(scores), axis=1)
    accuracy = np.sum(y_prediction == y_train) / float(len(y_train))
        
    if loss < min_loss:
        min_loss = loss
        best_weights = (W_1, b_1, W_2, b_2)
        print 'loss %s accuracy %s loop %s' % (round(loss, 3), round(accuracy, 3), i)

print '\ntime taken %s seconds' % str(time() - start)

loss 0.688 accuracy 0.597 loop 0
loss 0.679 accuracy 0.654 loop 1
loss 0.677 accuracy 0.625 loop 15
loss 0.675 accuracy 0.61 loop 24
loss 0.675 accuracy 0.699 loop 32
loss 0.673 accuracy 0.732 loop 56
loss 0.66 accuracy 0.732 loop 88
loss 0.659 accuracy 0.671 loop 352

time taken 11.8859460354 seconds


In [9]:
W_1, b_1, W_2, b_2 = best_weights
scores = []

for j in xrange(X_test.shape[0]):
    result = np.dot(W_1, X_test[j]) + b_1
    result = np.dot(W_2, result) + b_2
    result = softmax(result)
    scores.append(list(result))
    
y_prediction = np.argmax(np.array(scores), axis=1)

print "accuracy", np.sum(y_prediction == y_test) / float(len(y_test))

accuracy 0.703910614525


In [10]:
min_loss = 1000000
best_weights = ()

start = time()

for i in xrange(1000):
    W_1 = np.random.rand(100, 6) / 10
    b_1 = np.random.rand(100,) / 10
    W_2 = np.random.rand(100, 100) / 10
    b_2 = np.random.rand(100,) / 10
    W_3 = np.random.rand(2, 100) / 10
    b_3 = np.random.rand(2,) / 10
    
    scores = []
    loss = 0

    for j in xrange(X_train.shape[0]):
        result = np.dot(W_1, X_train[j]) + b_1
        result = np.dot(W_2, result) + b_2
        result = np.dot(W_3, result) + b_3
        result = softmax(result)
        scores.append(list(result))
        
        label_index = np.argmax(y_train_onehot[j])
        loss += -np.log(result[label_index])
        
    loss = loss / float(X_train.shape[0])
    y_prediction = np.argmax(np.array(scores), axis=1)
    accuracy = np.sum(y_prediction == y_train) / float(len(y_train))          
        
    if loss < min_loss:
        min_loss = loss
        best_weights = (W_1, b_1, W_2, b_2, W_3, b_3)
        print 'loss %s accuracy %s loop %s' % (round(loss, 3), round(accuracy, 3), i)

print '\ntime taken %s seconds' % str(time() - start)

loss 0.724 accuracy 0.281 loop 0
loss 0.68 accuracy 0.642 loop 1
loss 0.66 accuracy 0.718 loop 2
loss 0.634 accuracy 0.699 loop 8
loss 0.621 accuracy 0.718 loop 19
loss 0.616 accuracy 0.718 loop 114
loss 0.615 accuracy 0.73 loop 156
loss 0.604 accuracy 0.729 loop 316

time taken 16.2857968807 seconds


In [11]:
W_1, b_1, W_2, b_2, W_3, b_3 = best_weights
scores = []

for j in xrange(X_test.shape[0]):
    result = np.dot(W_1, X_test[j]) + b_1
    result = np.dot(W_2, result) + b_2
    result = np.dot(W_3, result) + b_3    
    result = softmax(result)
    scores.append(list(result))
    
y_prediction = np.argmax(np.array(scores), axis=1)

print "accuracy", np.sum(y_prediction == y_test) / float(len(y_test))

accuracy 0.703910614525
